In [288]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import tensorflow as tf

In [289]:
df=pd.read_csv('clean_data/training_data.csv')

In [290]:
df.drop('Unnamed: 0',axis=1,inplace=True)
#train_data=df[(df['G1.H']!='0')&(df['G2.H']!='0')&(df['G3.H']!='0')&(df['G1.A']!='0')&(df['G2.A']!='0')&(df['G2.A']!='0')&(df['G3.A']!='0')]
train_data=df
train_data.index=range(0,len(train_data))

In [291]:
for i in range(1,4):
    train_data[['G0','G'+str(i)+'.H.D','G'+str(i)+'.H.L','G'+str(i)+'.H.W']]=pd.get_dummies(train_data['G'+str(i)+'.H'])
    train_data[['G0','G'+str(i)+'.A.D','G'+str(i)+'.A.L','G'+str(i)+'.A.W']]=pd.get_dummies(train_data['G'+str(i)+'.A'])

In [292]:
train_data.columns

Index(['Home_Points', 'Away_Points', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG',
       'FTAG', 'Home_Goals', 'Away_Goals', 'Home_Goals_Conceded',
       'Away_Goals_Conceded', 'Form_Rating_Home', 'Form_Rating_Away',
       'Home_Shots', 'Away_Shots', 'Home_Shots_Target', 'Away_Shots_Target',
       'Home_Corners', 'Away_Corners', 'Home_Yellow', 'Away_Yellow',
       'Home_Red', 'Away_Red', 'G1.H', 'G2.H', 'G3.H', 'G1.A', 'G2.A', 'G3.A',
       'This_Year_LP_Home', 'Last_Year_LP_Home', 'This_Year_LP_Away',
       'Last_Year_LP_Away', 'TY_Points_Home', 'TY_Points_Away',
       'TY_Goals_Home', 'TY_Goals_Away', 'TY_Goals_Conceded_Home',
       'TY_Goals_Conceded_Away', 'TY_Shots_Home', 'TY_Shots_Away',
       'TY_Shots_On_Target_Home', 'TY_Shot_On_Target_Away', 'FTR', 'Home_GD',
       'Away_GD', 'Team_Caliber_Home', 'Team_Caliber_Away', 'G0', 'G1.H.D',
       'G1.H.L', 'G1.H.W', 'G1.A.D', 'G1.A.L', 'G1.A.W', 'G2.H.D', 'G2.H.L',
       'G2.H.W', 'G2.A.D', 'G2.A.L', 'G2.A.W', 'G3.H.D', 'G3.H.

In [293]:
def result_mapper(result):
    if result=='H':
        return 2
    elif result=='A':
        return 1
    else:
        return 0

In [294]:
def reverse_map(result):
    if result==2:
        return 'H'
    elif result==1:
        return 'A'
    else:
        return 'D'

In [295]:
def change_result(val):
    if val=='W':
        return 1
    elif val=='L':
        return -1
    else:
        return 0

In [296]:
def isDraw(val):
    if val=='D':
        return 1
    else:
        return 0

In [297]:
train_data['FTR.numerical']=train_data['FTR'].apply(result_mapper)
train_data['TYD']=abs(train_data['This_Year_LP_Home']-train_data['This_Year_LP_Away'])
train_data['LYD']=abs(train_data['Last_Year_LP_Home']-train_data['Last_Year_LP_Away'])
train_data['isDraw']=train_data['FTR'].apply(isDraw)

In [298]:
for i in range(1,4):

    train_data['G'+str(i)+'.H']=train_data['G'+str(i)+'.H'].apply(change_result)
    train_data['G'+str(i)+'.A']=train_data['G'+str(i)+'.A'].apply(change_result)

In [299]:
x=train_data[['Home_Points', 'Away_Points', 'Home_GD', 'Away_GD','G1.H.D',
       'G1.H.L', 'G1.H.W', 'G1.A.D', 'G1.A.L', 'G1.A.W', 'G2.H.D', 'G2.H.L',
       'G2.H.W', 'G2.A.D', 'G2.A.L', 'G2.A.W', 'G3.H.D', 'G3.H.L', 'G3.H.W',
       'G3.A.D', 'G3.A.L', 'G3.A.W', 'Home_Shots', 'Away_Shots','Home_Red', 'Away_Red','This_Year_LP_Home',
       'Last_Year_LP_Home', 'This_Year_LP_Away', 'Last_Year_LP_Away']]
y=train_data['FTR.numerical']

In [300]:
y=np.asarray(y).reshape(len(y),1)

In [301]:
from sklearn.preprocessing import StandardScaler

In [302]:
scale=StandardScaler()

In [303]:
scale.fit(x)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [304]:
scaled_data=scale.transform(x)

In [305]:
#Y=tf.keras.utils.to_categorical(np.asarray(pd.get_dummies(y)))

In [306]:
from sklearn.model_selection import train_test_split

In [307]:
x_1=np.asarray(scaled_data).astype(float)
y_1=tf.keras.utils.to_categorical(y)

In [308]:
x_train,x_test,y_train,y_test=train_test_split(x_1,y,test_size=0.30,random_state=101)

In [309]:
#Creating Neural Network Model, with hyper parameter tuning 
import tensorflow as tf
from keras.models import Sequential 
from keras.layers import Dense,Dropout,LSTM,BatchNormalization
from keras.callbacks import TensorBoard 
from keras.optimizers import adam,rmsprop,SGD
from keras.regularizers import l1
from tensorflow.keras.callbacks import EarlyStopping

In [310]:
def create_model(learning_rate,num_dense_layers,num_dense_nodes,num_classes,num_features,momentum,decay):
    
    #create model
    
    model=Sequential()
    
    model.add(Dense(num_features,activation='relu',kernel_initializer=tf.keras.initializers.glorot_uniform(seed=None)))
    model.add(Dropout(0.20))
    #reg = l1(0.001)
    
    for i in range(num_dense_layers):
        name='layer_dense_{0}'.format(i+1)
        model.add(Dense(num_dense_nodes,activation='relu',name=name))
        #model.add(Dropout(0.50))
        
    model.add(Dense(num_classes,activation='softmax',activity_regularizer=l1(0.01)))
   # model.summary()
    #Optimizer=adam(lr=learning_rate,beta_1=1,beta_2=0.999)
    Optimizer=SGD(lr=learning_rate,momentum=momentum,decay=decay)
    
   
    
    model.compile(optimizer=Optimizer,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    return model

In [311]:
stopping=EarlyStopping(monitor='loss',min_delta=1e-4,patience=20,verbose=1,mode='auto')

In [312]:
model=create_model(learning_rate=0.004,num_dense_layers=4,num_dense_nodes=30,num_classes=3,num_features=len(x.columns),momentum=0.25,decay=0.0001)

In [313]:
model.fit(x_train,tf.keras.utils.to_categorical(y_train),validation_data=(x_test,tf.keras.utils.to_categorical(y_test)),batch_size=31,epochs=200,callbacks=[stopping])

Train on 2660 samples, validate on 1140 samples
Epoch 1/200
2660/2660 [==============================] - 1s 385us/step - loss: 1.4036 - acc: 0.4079 - val_loss: 1.3910 - val_acc: 0.4693
Epoch 2/200
2660/2660 [==============================] - 0s 69us/step - loss: 1.3843 - acc: 0.4632 - val_loss: 1.3775 - val_acc: 0.4693
Epoch 3/200
2660/2660 [==============================] - 0s 73us/step - loss: 1.3747 - acc: 0.4658 - val_loss: 1.3694 - val_acc: 0.4693
Epoch 4/200
2660/2660 [==============================] - 0s 76us/step - loss: 1.3692 - acc: 0.4643 - val_loss: 1.3634 - val_acc: 0.4693
Epoch 5/200
2660/2660 [==============================] - 0s 78us/step - loss: 1.3628 - acc: 0.4650 - val_loss: 1.3588 - val_acc: 0.4693
Epoch 6/200
2660/2660 [==============================] - 0s 82us/step - loss: 1.3602 - acc: 0.4643 - val_loss: 1.3549 - val_acc: 0.4693
Epoch 7/200
2660/2660 [==============================] - 0s 80us/step - loss: 1.3565 - acc: 0.4643 - val_loss: 1.3511 - val_acc: 0.4693

2660/2660 [==============================] - 0s 80us/step - loss: 1.2933 - acc: 0.5297 - val_loss: 1.2980 - val_acc: 0.5298
Epoch 61/200
2660/2660 [==============================] - 0s 68us/step - loss: 1.2857 - acc: 0.5357 - val_loss: 1.2981 - val_acc: 0.5316
Epoch 62/200
2660/2660 [==============================] - 0s 86us/step - loss: 1.2854 - acc: 0.5376 - val_loss: 1.2979 - val_acc: 0.5281
Epoch 63/200
2660/2660 [==============================] - 0s 87us/step - loss: 1.2884 - acc: 0.5380 - val_loss: 1.2978 - val_acc: 0.5289
Epoch 64/200
2660/2660 [==============================] - 0s 107us/step - loss: 1.2883 - acc: 0.5305 - val_loss: 1.2976 - val_acc: 0.5289
Epoch 65/200
2660/2660 [==============================] - 0s 98us/step - loss: 1.2896 - acc: 0.5350 - val_loss: 1.2974 - val_acc: 0.5281
Epoch 66/200
2660/2660 [==============================] - 0s 99us/step - loss: 1.2836 - acc: 0.5357 - val_loss: 1.2975 - val_acc: 0.5272
Epoch 67/200
2660/2660 [=============================

2660/2660 [==============================] - 0s 84us/step - loss: 1.2737 - acc: 0.5387 - val_loss: 1.2947 - val_acc: 0.5342
Epoch 120/200
2660/2660 [==============================] - 0s 105us/step - loss: 1.2794 - acc: 0.5365 - val_loss: 1.2946 - val_acc: 0.5342
Epoch 121/200
2660/2660 [==============================] - 0s 104us/step - loss: 1.2783 - acc: 0.5421 - val_loss: 1.2945 - val_acc: 0.5342
Epoch 122/200
2660/2660 [==============================] - 0s 107us/step - loss: 1.2780 - acc: 0.5414 - val_loss: 1.2945 - val_acc: 0.5342
Epoch 123/200
2660/2660 [==============================] - 0s 107us/step - loss: 1.2797 - acc: 0.5376 - val_loss: 1.2944 - val_acc: 0.5342
Epoch 124/200
2660/2660 [==============================] - 0s 104us/step - loss: 1.2788 - acc: 0.5410 - val_loss: 1.2944 - val_acc: 0.5342
Epoch 125/200
2660/2660 [==============================] - 0s 110us/step - loss: 1.2795 - acc: 0.5372 - val_loss: 1.2945 - val_acc: 0.5342
Epoch 126/200
2660/2660 [=================

2660/2660 [==============================] - 0s 82us/step - loss: 1.2746 - acc: 0.5444 - val_loss: 1.2940 - val_acc: 0.5351
Epoch 179/200
2660/2660 [==============================] - 0s 94us/step - loss: 1.2705 - acc: 0.5455 - val_loss: 1.2940 - val_acc: 0.5351
Epoch 180/200
2660/2660 [==============================] - 0s 96us/step - loss: 1.2731 - acc: 0.5477 - val_loss: 1.2939 - val_acc: 0.5351
Epoch 181/200
2660/2660 [==============================] - 0s 102us/step - loss: 1.2713 - acc: 0.5429 - val_loss: 1.2939 - val_acc: 0.5342
Epoch 182/200
2660/2660 [==============================] - 0s 96us/step - loss: 1.2735 - acc: 0.5455 - val_loss: 1.2939 - val_acc: 0.5351
Epoch 183/200
2660/2660 [==============================] - 0s 79us/step - loss: 1.2750 - acc: 0.5387 - val_loss: 1.2940 - val_acc: 0.5342
Epoch 184/200
2660/2660 [==============================] - 0s 98us/step - loss: 1.2740 - acc: 0.5492 - val_loss: 1.2939 - val_acc: 0.5360
Epoch 185/200
2660/2660 [======================

In [372]:
model_json=model.to_json()

with open("model.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

print("Saved to disk")

Saved to disk


In [314]:
predictions=model.predict(x_test)

In [315]:
predict=[]
for i in predictions:
    predict.append(reverse_map(np.argmax(i)))
real=[]     
for i in y_test:
    real.append(reverse_map(i))

In [316]:
from sklearn.metrics import classification_report,confusion_matrix

In [317]:
print(classification_report(real,predict))

             precision    recall  f1-score   support

          A       0.51      0.51      0.51       324
          D       0.33      0.01      0.01       281
          H       0.55      0.83      0.66       535

avg / total       0.48      0.53      0.46      1140



In [354]:
predictions

array([[0.22028664, 0.570033  , 0.20968033],
       [0.21354742, 0.59243834, 0.19401431],
       [0.23883967, 0.18257914, 0.5785812 ],
       ...,
       [0.25518274, 0.25317338, 0.49164385],
       [0.22485955, 0.5509195 , 0.22422096],
       [0.29603583, 0.3057378 , 0.39822644]], dtype=float32)

In [355]:
'''

Possible Improvements:

1.Add League Position Input (X) 
2.Do point Totals for Each Season in EPL
3.Do Last Season Point Total and This Season Point Total Features (X-For League Positions)

'''

'\n\nPossible Improvements:\n\n1.Add League Position Input (X) \n2.Do point Totals for Each Season in EPL\n3.Do Last Season Point Total and This Season Point Total Features (X-For League Positions)\n\n'

In [356]:
from sklearn.naive_bayes import GaussianNB

In [357]:
gnb=GaussianNB()

In [358]:
gnb.fit(x_train,y_train)

/home/addybrown/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None)

In [359]:
gaussian_preds=[]

for i in gnb.predict(x_test):
    gaussian_preds.append(reverse_map(i))

In [360]:
print(classification_report(gaussian_preds,test_results))

             precision    recall  f1-score   support

          A       0.52      0.46      0.49       370
          D       0.06      0.29      0.10        55
          H       0.74      0.55      0.63       715

avg / total       0.63      0.51      0.56      1140



In [361]:
gnb.score(x_test,y_test)

0.5087719298245614

In [362]:
from sklearn.naive_bayes import BernoulliNB

In [363]:
bnb=BernoulliNB()

In [364]:
bnb.fit(x_train,y_train)

/home/addybrown/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [365]:
bnb.score(x_test,y_test)

0.5131578947368421

In [366]:
test_results=[]
for i in y_test:
    test_results.append(reverse_map(i))

In [367]:
bernoulli_preds=[]

for i in bnb.predict(x_test):
    bernoulli_preds.append(reverse_map(i))

In [368]:
print(classification_report(bernoulli_preds,test_results))

             precision    recall  f1-score   support

          A       0.56      0.46      0.50       398
          D       0.07      0.30      0.11        64
          H       0.72      0.57      0.63       678

avg / total       0.63      0.51      0.56      1140



In [369]:
bnb.score(x_test,y_test)

0.5131578947368421